# This notebook helps to create the B tree in form of a table

### First createing a dict with each word across all the con files and taking its count

In [42]:
import pandas as pd
import numpy as np
import glob
import re
from nltk.corpus import stopwords
import string
import operator
import itertools

In [2]:
d = dict()
stop = stopwords.words('english') + list(string.punctuation)

In [3]:
for filename in glob.glob('samples/*.con'):
#     print("****************************************************************")
#     print(filename)
#     print("****************************************************************")
    with open(filename) as f:
        for line in f:
            if not line:
                pass
            else:
                if len(line.split("\"")) == 5:
                    temp = line.split("\"")[1]
                    x = temp.translate(None, string.punctuation)
                    temp = x.strip()
        #             temp_arr = [i for i in temp.split() if i not in stop]
                    if temp in d:
                        d[temp] += 1

                    else:
                        d[temp] = 0
                        d[temp] += 1
            


In [4]:
df = pd.DataFrame.from_dict(d, orient='index')
df.columns=['Values']

In [5]:
df = df.sort_values(by=['Values'], ascending=False)

In [6]:
df.head()

,Values
type num diabetes,321
type num diabetes mellitus,306
treatment,267
the study,245
insulin,233


In [110]:
df.drop(df.head(4).index, inplace=True)

In [9]:
df.head

<bound method DataFrame.head of                                                     Values
type num diabetes                                      321
type num diabetes mellitus                             306
treatment                                              267
the study                                              245
insulin                                                233
metformin                                              224
type 2 diabetes mellitus                               178
diabetes                                               162
type 2 diabetes                                        158
type 1 diabetes                                        145
hba1c 0                                                140
hba1c 5                                                138
type 1 diabetes mellitus                               118
body mass index bmi                                     95
hemoglobin                                              89
medications             

### Start creating the table

In [3]:
lst = list(itertools.product([0, 1], repeat=6))

In [4]:
table = {}
for i in lst:
    table[i] = []

In [5]:
table

{(0, 0, 0, 0, 0, 0): [],
 (0, 0, 0, 0, 0, 1): [],
 (0, 0, 0, 0, 1, 0): [],
 (0, 0, 0, 0, 1, 1): [],
 (0, 0, 0, 1, 0, 0): [],
 (0, 0, 0, 1, 0, 1): [],
 (0, 0, 0, 1, 1, 0): [],
 (0, 0, 0, 1, 1, 1): [],
 (0, 0, 1, 0, 0, 0): [],
 (0, 0, 1, 0, 0, 1): [],
 (0, 0, 1, 0, 1, 0): [],
 (0, 0, 1, 0, 1, 1): [],
 (0, 0, 1, 1, 0, 0): [],
 (0, 0, 1, 1, 0, 1): [],
 (0, 0, 1, 1, 1, 0): [],
 (0, 0, 1, 1, 1, 1): [],
 (0, 1, 0, 0, 0, 0): [],
 (0, 1, 0, 0, 0, 1): [],
 (0, 1, 0, 0, 1, 0): [],
 (0, 1, 0, 0, 1, 1): [],
 (0, 1, 0, 1, 0, 0): [],
 (0, 1, 0, 1, 0, 1): [],
 (0, 1, 0, 1, 1, 0): [],
 (0, 1, 0, 1, 1, 1): [],
 (0, 1, 1, 0, 0, 0): [],
 (0, 1, 1, 0, 0, 1): [],
 (0, 1, 1, 0, 1, 0): [],
 (0, 1, 1, 0, 1, 1): [],
 (0, 1, 1, 1, 0, 0): [],
 (0, 1, 1, 1, 0, 1): [],
 (0, 1, 1, 1, 1, 0): [],
 (0, 1, 1, 1, 1, 1): [],
 (1, 0, 0, 0, 0, 0): [],
 (1, 0, 0, 0, 0, 1): [],
 (1, 0, 0, 0, 1, 0): [],
 (1, 0, 0, 0, 1, 1): [],
 (1, 0, 0, 1, 0, 0): [],
 (1, 0, 0, 1, 0, 1): [],
 (1, 0, 0, 1, 1, 0): [],
 (1, 0, 0, 1, 1, 1): [],


### Now that we have the table, loading the inclusion_exclusion.csv and searching for the top keywords in each file

In [2]:
inc_ex_df = pd.read_csv('inclusion_exclusion.csv')

In [3]:
inc_ex_df = inc_ex_df.set_index('NCT')

In [4]:
inc_ex_df = inc_ex_df.drop(['Unnamed: 0'], axis=1)

In [5]:
len(inc_ex_df)

904

In [6]:
inc_ex_df.head()

,Exclusion,Inclusion
NCT,,
NCT02325466,: - Subject is pregnant or breast-...,: - Female or male aged ≥ 35 years...
NCT00555217,: - History of intolerance to ACEI...,: - Type 2 diabetes - ...
NCT03211858,NaN,I
NCT02429258,: - For patients who enter the stu...,: - Type 2 diabetes mellitus (T2DM...
NCT01106625,: - History of diabetic ketoacidos...,: - All patients must have a diagn...


### Now search in inclusion and exclusion for 3 features
### Features: hypertension, insulin, metformin 
#### all features have following convention: 0 if not in inclusion or if in exclusion, 1 otherwise

In [7]:
f = pd.DataFrame(columns=['hypertension', 'metformin', 'insulin'])

In [8]:
def findVal(word, criteria):
    if word in criteria.lower():
        return 1
    return 0

In [9]:
import csv

words = ["hypertension", "metformin", "insulin"]
store = dict()
with open("inclusion_exclusion.csv", "r") as file:
    reader = csv.reader(file)
    for row in reader:
        result =[]
        for word in words:
            result.append(findVal(word, row[2]))
        store[row[3]]=result

In [10]:
del(store['NCT'])

### Store dict holds the required table

## Finding out the ranges for bmi

In [63]:
bmi_dict = {}

In [64]:
for filename in glob.glob('samples/*.con'):
#     print("****************************************************************")
#     print(filename)
#     print("****************************************************************")
    with open(filename) as f:
        for line in f:
            if not line:
                pass
            else:
                bmi_line = re.findall(r'bmi [0-9]*', line)
                if bmi_line:
                    # Here we have all those string which have bmi and some value in them
                    digit = re.findall(r'\d+', bmi_line[0])
                    if digit:
                        digit = int(digit[0])
#                     print(digit)
                        name = filename.split("/")
                        name = name[1].split(".")[0]
                        bmi_dict.update({name:digit})
#                     digit = int(filter(str.isdigit, bmi_line[0]))
#                     print(digit)

In [65]:
bmi_dict

{'NCT00015626': 1,
 'NCT00017953': 25,
 'NCT00029848': 27,
 'NCT00035984': 27,
 'NCT00039013': 27,
 'NCT00039026': 27,
 'NCT00058981': 40,
 'NCT00099320': 25,
 'NCT00104182': 40,
 'NCT00111540': 25,
 'NCT00135330': 25,
 'NCT00147719': 45,
 'NCT00147758': 45,
 'NCT00196989': 25,
 'NCT00212641': 4,
 'NCT00220961': 135,
 'NCT00239538': 7,
 'NCT00240370': 41,
 'NCT00294723': 45,
 'NCT00303134': 7,
 'NCT00308139': 25,
 'NCT00308308': 35,
 'NCT00309244': 40,
 'NCT00322257': 40,
 'NCT00328172': 6,
 'NCT00332488': 40,
 'NCT00332826': 40,
 'NCT00333151': 45,
 'NCT00386100': 25,
 'NCT00391196': 27,
 'NCT00399711': 45,
 'NCT00406458': 38,
 'NCT00411892': 40,
 'NCT00425919': 23,
 'NCT00427154': 40,
 'NCT00467649': 25,
 'NCT00474630': 27,
 'NCT00476931': 38,
 'NCT00478322': 2,
 'NCT00484198': 45,
 'NCT00495469': 22,
 'NCT00502710': 25,
 'NCT00509223': 4,
 'NCT00510952': 25,
 'NCT00518882': 45,
 'NCT00535535': 25,
 'NCT00547482': 1,
 'NCT00553787': 27,
 'NCT00565799': 35,
 'NCT00596973': 3,
 'NCT005

In [66]:
bmi_dataframe = pd.DataFrame.from_dict(bmi_dict, orient='index')

In [68]:
bmi_dataframe.head()

,0
NCT00411892,40
NCT00947557,20
NCT01165684,40
NCT03060980,25
NCT02580591,18
